# Overview
this is a training script for the RNN. To train against multiple types of agents, edit the run_game method.
In order for this program to work record_data must be working

In [1]:
import torch
import pickle
from uncertainty_rnn import BoardGuesserNet
import torch.optim as optim
import numpy as np
import time
import os
import matplotlib.pyplot as plt
from collections import deque
import sys


# Board Guess

In [2]:
# set up Board Guesser Net
guessNet = BoardGuesserNet()

#set up loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(guessNet.parameters(), lr = .001)

# Constants
train_iterations = 100000
validation_count  = 50 # after every 20 games check validation score
load_in_weights = False

C:\Users\Michael Jurado\workspace\ReconChess\Another_Chess_Bot\uncertainty_rnn.py:12: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.conv1.weight, gain = 1)
C:\Users\Michael Jurado\workspace\ReconChess\Another_Chess_Bot\uncertainty_rnn.py:17: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.conv2.weight, gain = 1)
C:\Users\Michael Jurado\workspace\ReconChess\Another_Chess_Bot\uncertainty_rnn.py:27: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.dense1.weight, gain = 1)


# Definer helpers

In [3]:

def save_model(guessNet):
    # definer helpers
    try:
        os.mkdir("rnn_model")
    except Exception as e:
        print(e)

    torch.save(guessNet.state_dict(), "rnn_model")
    
def load_model(guessNet):
    guessNet.load_state_dict(torch.load("rnn_model"))
    
def run_game():
    """
    Returns X_train_batch, y_train_batch, which are both numpy arrays
    
    NOTE: Addit command to change which agent plays. (Must save data in specified format though)
    """
    start = time.time()
    !python modified_play_game.py random_agent_save_obs random_agent_save_obs
    print("time to run game", time.time() - start)
    
    # now lets load the data in. Randomly alternate between players
    if np.random.randint(0, 2) == 0:
        in_ = open("white_game_obs.pkl", "rb")
    else:
        in_ = open("black_game_obs.pkl", "rb")
        
    player_sense_list, player_truth_board_list = pickle.load(in_)
    in_.close()
    
    X_train_batch = np.array(player_sense_list)
    y_train_batch = np.array(player_truth_board_list)
    return X_train_batch, y_train_batch


def record_game_loss(pred_list, y_train_batch):
    """
    Record mse between truth board and predictions. 
    THis is for validation
    """
    if isinstnace(pred_list, torch.Tensor):
        y_train_batch = y_train_batch.detach().cpu().numpy()
        pred_list = pred_list.detach().cpu().numpy()
    return np.sum((pred_list - y_train_batch)**2)



    
def create_loss_plot(train_loss_history):
    plt.figure()

    x_axis = (np.arange(len(train_loss_history)))
    
    plt.plot(x_axis, train_loss_history, label = "train_loss")
    
    plt.legend(loc="upper left")
    plt.ylabel("MSE")
    plt.xlabel("epochs")    
    try:
        plt.savefig("rnn_loss.png")
    except Exception as e:
        print("Permission denied saving loss")
        
    plt.cla()
    plt.clf()
    plt.close()
    
    

# Training Step

In [ ]:
if load_in_weights:
    load_model(guessNet)
    # TODO

# contains train and test lost history
train_loss_history = []
test_loss_history = []
plt.figure()

train_loss_queue = deque(maxlen = 100)  # holds a temporary queue of train lost
smallestLoss = sys.maxsize
lastImprovement = 0
for epoch in range(train_iterations):
    X_train_batch, y_train_batch = run_game() # plays a game random versus random

    X_train_batch = torch.Tensor(X_train_batch)
    # training step
    pred_labels = guessNet(X_train_batch)
    y_train_batch = torch.Tensor(y_train_batch)
    loss = criterion(pred_labels, y_train_batch)
    
    loss.backward()
    optimizer.step() # magic    
    train_loss_queue.append(loss.detach().cpu().numpy())
    
    loss = np.mean(train_loss_queue)
    train_loss_history.append(np.mean(train_loss_queue))

    if (epoch +1) % 5 == 0:  # save loss plot every 5 steps
        create_loss_plot(train_loss_history)
        
    # run a validation every 100 steps
    if (epoch+ 1) % 100 == 0:
        print("validation time")
        total_val_loss = 0
        for i in range(validation_count):
            X_train_batch, y_train_batch = run_game() # plays a game random versus random
            X_train_batch = torch.Tensor(X_train_batch)
            
            # training step
            pred_labels = guessNet(X_train_batch)
            y_train_batch = torch.Tensor(y_train_batch)
            total_val_loss += criterion(pred_labels, y_train_batch)   
        val_loss = total_val_loss/ validation_count
        
        if val_loss < smallestLoss:
            print("better model found with loss", val_loss)
            save_model(guessNet)
            smallestLoss =  val_loss
    
    # decay learning rate
    if (epoch+1) % 200 == 0:
        for param_group in optimizer.param_groups:
                param_group['lr'] *= .98     
                param_group['lr'] = max(.0001, param_group['lr'])
        print("decaying learning rate to", param_group["lr"])
        
    # print out pred_board[0]
    if (epoch+1) % 100 == 0:
        print("pred board for first time step")
        index = 0
        for channel in [np.argwhere(i > .5) for i in pred_labels[0].detach().numpy()]:
            print("active arguments in channel", index)
            print(channel)
            index += 1
    
        

            

Game Over!
WHITE won by king capture.
time to run game 1.7048068046569824
Game Over!
BLACK won by king capture.
time to run game 0.9068272113800049
Game Over!
BLACK won by king capture.
time to run game 1.044274091720581
Game Over!
WHITE won by king capture.
time to run game 0.8168652057647705
Game Over!
BLACK won by king capture.
time to run game 0.9267616271972656
Game Over!
BLACK won by king capture.
time to run game 0.8385810852050781
Game Over!
BLACK won by king capture.
time to run game 0.770587682723999
Game Over!
WHITE won by king capture.
time to run game 1.0250179767608643
Game Over!
WHITE won by king capture.
time to run game 1.1080076694488525
Game Over!
BLACK won by king capture.
time to run game 0.8756086826324463
Game Over!
WHITE won by king capture.
time to run game 0.7981512546539307
Game Over!
WHITE won by king capture.
time to run game 0.8627238273620605
Game Over!
WHITE won by king capture.
time to run game 0.8438937664031982
Game Over!
WHITE won by king capture.
ti

Game Over!
BLACK won by king capture.
time to run game 0.8037450313568115
Game Over!
BLACK won by king capture.
time to run game 0.9178345203399658
Game Over!
WHITE won by king capture.
time to run game 0.721656084060669
Game Over!
BLACK won by king capture.
time to run game 0.9228386878967285
Game Over!
BLACK won by king capture.
time to run game 0.9960370063781738
Game Over!
BLACK won by king capture.
time to run game 0.7671592235565186
Game Over!
BLACK won by king capture.
time to run game 0.8427960872650146
Game Over!
BLACK won by king capture.
time to run game 0.7755470275878906
Game Over!
WHITE won by king capture.
time to run game 0.9410159587860107
Game Over!
BLACK won by king capture.
time to run game 1.244133710861206
Game Over!
WHITE won by king capture.
time to run game 0.7474932670593262
Game Over!
WHITE won by king capture.
time to run game 0.8107373714447021
Game Over!
BLACK won by king capture.
time to run game 0.7316653728485107
Game Over!
BLACK won by king capture.
ti

Game Over!
BLACK won by king capture.
time to run game 0.9619054794311523
Game Over!
BLACK won by king capture.
time to run game 0.9820072650909424
Game Over!
WHITE won by king capture.
time to run game 1.0643365383148193
Game Over!
WHITE won by king capture.
time to run game 0.9952905178070068
Game Over!
WHITE won by king capture.
time to run game 1.1906347274780273
Game Over!
BLACK won by king capture.
time to run game 1.417792797088623
Game Over!
WHITE won by king capture.
time to run game 0.8812134265899658
Game Over!
BLACK won by king capture.
time to run game 0.9771676063537598
Game Over!
BLACK won by king capture.
time to run game 0.9213426113128662
Game Over!
BLACK won by king capture.
time to run game 1.0004136562347412
Game Over!
BLACK won by king capture.
time to run game 0.9994187355041504
Game Over!
WHITE won by king capture.
time to run game 1.0966081619262695
Game Over!
WHITE won by king capture.
time to run game 0.7973589897155762
Game Over!
BLACK won by king capture.
t

Game Over!
BLACK won by king capture.
time to run game 0.9313507080078125
Game Over!
WHITE won by king capture.
time to run game 0.6325759887695312
Game Over!
BLACK won by king capture.
time to run game 0.8567790985107422
Game Over!
WHITE won by king capture.
time to run game 0.68662428855896
Game Over!
WHITE won by king capture.
time to run game 1.151047945022583
Game Over!
WHITE won by king capture.
time to run game 0.8327596187591553
Game Over!
BLACK won by king capture.
time to run game 0.8753006458282471
Game Over!
BLACK won by king capture.
time to run game 0.8507857322692871
Game Over!
WHITE won by king capture.
time to run game 1.4097132682800293
Game Over!
WHITE won by king capture.
time to run game 1.0569610595703125
Game Over!
BLACK won by king capture.
time to run game 0.7226564884185791
Game Over!
WHITE won by king capture.
time to run game 0.7411782741546631
Game Over!
WHITE won by king capture.
time to run game 0.8187441825866699
Game Over!
BLACK won by king capture.
tim

Game Over!
BLACK won by king capture.
time to run game 0.8377621173858643
Game Over!
BLACK won by king capture.
time to run game 0.8317549228668213
Game Over!
BLACK won by king capture.
time to run game 0.694631814956665
Game Over!
BLACK won by king capture.
time to run game 0.8057324886322021
Game Over!
WHITE won by king capture.
time to run game 0.8007278442382812
Game Over!
BLACK won by king capture.
time to run game 0.6606040000915527
Game Over!
WHITE won by king capture.
time to run game 0.9798922538757324
Game Over!
BLACK won by king capture.
time to run game 1.0539565086364746
Game Over!
WHITE won by king capture.
time to run game 1.1325325965881348
Game Over!
WHITE won by king capture.
time to run game 1.1965906620025635
Game Over!
WHITE won by king capture.
time to run game 0.8657867908477783
Game Over!
BLACK won by king capture.
time to run game 0.8007311820983887
Game Over!
WHITE won by king capture.
time to run game 0.9448587894439697
Game Over!
WHITE won by king capture.
t

...

Game Over!
WHITE won by king capture.
time to run game 0.7757244110107422
Game Over!
BLACK won by king capture.
time to run game 0.9343574047088623
Game Over!
WHITE won by king capture.
time to run game 0.8606152534484863
Game Over!
WHITE won by king capture.
time to run game 0.9312045574188232
Game Over!
BLACK won by king capture.
time to run game 0.8846340179443359
Game Over!
BLACK won by king capture.
time to run game 0.9729375839233398
Game Over!
WHITE won by king capture.
time to run game 0.6716115474700928
Game Over!
WHITE won by king capture.
time to run game 1.6547720432281494
Game Over!
BLACK won by king capture.
time to run game 0.7039072513580322
Game Over!
WHITE won by king capture.
time to run game 0.7730157375335693
Game Over!
BLACK won by king capture.
time to run game 2.079951763153076
Game Over!
BLACK won by king capture.
time to run game 0.8661794662475586
Game Over!
WHITE won by king capture.
time to run game 0.758275032043457
Game Over!
WHITE won by king capture.
ti

Game Over!
BLACK won by king capture.
time to run game 0.9463655948638916
Game Over!
WHITE won by king capture.
time to run game 0.7296640872955322
Game Over!
BLACK won by king capture.
time to run game 0.805931806564331
Game Over!
BLACK won by king capture.
time to run game 0.8283216953277588
Game Over!
WHITE won by king capture.
time to run game 0.8237030506134033
Game Over!
BLACK won by king capture.
time to run game 1.4918856620788574
Game Over!
WHITE won by king capture.
time to run game 1.2864453792572021
Game Over!
WHITE won by king capture.
time to run game 1.0829849243164062
Game Over!
WHITE won by king capture.
time to run game 0.9886157512664795
Game Over!
WHITE won by king capture.
time to run game 0.6130375862121582
Game Over!
BLACK won by king capture.
time to run game 0.94333815574646
Game Over!
WHITE won by king capture.
time to run game 0.7869081497192383
Game Over!
BLACK won by king capture.
time to run game 0.9525711536407471
Game Over!
WHITE won by king capture.
tim

Game Over!
WHITE won by king capture.
time to run game 1.047952651977539
Game Over!
BLACK won by king capture.
time to run game 0.8627851009368896
Game Over!
BLACK won by king capture.
time to run game 0.7937216758728027
Game Over!
WHITE won by king capture.
time to run game 0.6986351013183594
Game Over!
BLACK won by king capture.
time to run game 1.0709741115570068
Game Over!
BLACK won by king capture.
time to run game 0.7937214374542236
Game Over!
BLACK won by king capture.
time to run game 1.2146172523498535
Game Over!
BLACK won by king capture.
time to run game 1.0019104480743408
Game Over!
WHITE won by king capture.
time to run game 1.1010017395019531
Game Over!
BLACK won by king capture.
time to run game 0.8898091316223145
Game Over!
BLACK won by king capture.
time to run game 0.7917194366455078
Game Over!
WHITE won by king capture.
time to run game 0.9463632106781006
Game Over!
WHITE won by king capture.
time to run game 0.7426881790161133
Game Over!
BLACK won by king capture.
t

Game Over!
WHITE won by king capture.
time to run game 0.9768877029418945
Game Over!
WHITE won by king capture.
time to run game 0.9278435707092285
Game Over!
WHITE won by king capture.
time to run game 0.7431900501251221
Game Over!
BLACK won by king capture.
time to run game 0.9699296951293945
Game Over!
WHITE won by king capture.
time to run game 0.6921346187591553
Game Over!
BLACK won by king capture.
time to run game 0.7308580875396729
Game Over!
BLACK won by king capture.
time to run game 1.3570139408111572
Game Over!
BLACK won by king capture.
time to run game 0.7006490230560303
Game Over!
WHITE won by king capture.
time to run game 1.0680532455444336
Game Over!
WHITE won by king capture.
time to run game 1.0439496040344238
Game Over!
WHITE won by king capture.
time to run game 1.1095120906829834
Game Over!
WHITE won by king capture.
time to run game 0.989403247833252
Game Over!
WHITE won by king capture.
time to run game 0.7982392311096191
Game Over!
BLACK won by king capture.
t

Game Over!
BLACK won by king capture.
time to run game 1.0412497520446777
Game Over!
BLACK won by king capture.
time to run game 0.7146496772766113
Game Over!
WHITE won by king capture.
time to run game 0.8678827285766602
Game Over!
WHITE won by king capture.
time to run game 0.8267512321472168
Game Over!
BLACK won by king capture.
time to run game 0.7700636386871338
Game Over!
WHITE won by king capture.
time to run game 1.0257434844970703
Game Over!
BLACK won by king capture.
time to run game 0.8097193241119385
Game Over!
BLACK won by king capture.
time to run game 0.906987190246582
Game Over!
BLACK won by king capture.
time to run game 1.0324406623840332
Game Over!
BLACK won by king capture.
time to run game 0.6832275390625
Game Over!
WHITE won by king capture.
time to run game 0.8968265056610107
Game Over!
BLACK won by king capture.
time to run game 0.7727010250091553
Game Over!
BLACK won by king capture.
time to run game 1.249349594116211
Game Over!
BLACK won by king capture.
time 